In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import Adam
import numpy as np
import random
import io
from our_utilities import *

In [ ]:
# Data w/ poem numbers

path = 'data/shakespeare.txt'
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()

In [ ]:
# Pre-process data without poem numbers

sonnets, word2syll, word2index, index2word = loadsonnets(dictname='data/Syllable_dictionary.txt',
                      filename='data/shakespeare.txt')

In [ ]:
# Writing poems without poem numbering

f3 = open("data/shakespeare_no_number.txt", "w")

for i in range(len(sonnets)):
    sonnet = sonnets[i]
    for w in sonnet:
        if index2word[w] == '\\':
            f3.write('\n')
        else:
            f3.write(str(index2word[w]) + ' ')
    f3.write('\n')

f3.close()

In [ ]:
# Sonnet Generation

path = 'data/shakespeare_no_number.txt'
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
    
chars = sorted(list(set(text)))
chars.append('?')
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# semi-redundant sequences 
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
    
x = np.zeros((len(sentences), maxlen, len(chars)))
y = np.zeros((len(sentences), len(chars)))
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
    
model = Sequential()
model.add(LSTM(150, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [ ]:
model.fit(x, y, batch_size=128, epochs=25)

In [ ]:
def generate(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
# Seed sentence, temperature 1.5

generated = ''
sentence = "shall i compare thee to a summer's day?\n"
generated += sentence

for i in range(600):
    x_pred = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
        x_pred[0, t, char_indices[char]] = 1.

    preds = model.predict(x_pred, verbose=0)[0]
    next_index = generate(preds, 1.5)
    next_char = indices_char[next_index]

    generated += next_char
    sentence = sentence[1:] + next_char

In [ ]:
print(generated)

In [ ]:
# Seed sentence, temperature 0.75

generated = ''
sentence = "shall i compare thee to a summer's day?\n"
generated += sentence

for i in range(600):
    x_pred = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
        x_pred[0, t, char_indices[char]] = 1.

    preds = model.predict(x_pred, verbose=0)[0]
    next_index = generate(preds, 1.5)
    next_char = indices_char[next_index]

    generated += next_char
    sentence = sentence[1:] + next_char

In [ ]:
print(generated)

In [ ]:
# Seed sentence, temperature 0.25

generated = ''
sentence = "shall i compare thee to a summer's day?\n"
generated += sentence

for i in range(600):
    x_pred = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
        x_pred[0, t, char_indices[char]] = 1.

    preds = model.predict(x_pred, verbose=0)[0]
    next_index = generate(preds, 1.5)
    next_char = indices_char[next_index]

    generated += next_char
    sentence = sentence[1:] + next_char

In [ ]:
print(generated)